<a href="https://colab.research.google.com/github/hwang-sudang/kkoApplemusic/blob/sparse_matrix/0723_Autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Load

In [2]:
from google.colab import drive 
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [3]:
import os
os.chdir("/content/gdrive/My Drive/melon-playlist-continuation-master")
print(os.getcwd())
!ls

/content/gdrive/My Drive/melon-playlist-continuation-master
 0705Tags.ipynb			   Noising_func.ipynb
'0716 Khaiii main keyword.ipynb'   NOTICE.md
'0719 tag_sparse matrix.ipynb'	   PlaylistEmbedding.ipynb
'0720 tag_sparse_matrix.ipynb'	   __pycache__
 Apple_Music_Model_BPR.ipynb	   README.md
 arena_util.py			   requirements.txt
 build				   results.json
 evaluate.py			   song_meta.json
 genre_gn_all.json		  'sparse matrix.ipynb'
 genre_most_popular.py		   split_data.ipynb
 khaiii				   split_data.py
 LICENSE			   test.json
 LMF01_150_songs.json		   testPickleFile.pickle
 LMF01_50_songs.json		   train.json
 LMF01_70_songs.json		   uniq_tags.gsheet
 most_popular.py		   val.json


In [4]:
!pip install implicit==0.4.0

     |████████████████████████████████| 1.1MB 10.9MB/s 
  Created wheel for implicit: filename=implicit-0.4.0-cp36-cp36m-linux_x86_64.whl size=3390293 sha256=bf7e54fd088cefa5e00f395d9f6aee3c81a8975a26ab92009efe13df48837668
  Stored in directory: /root/.cache/pip/wheels/fd/d1/66/84e83191d9f626c521271023e4ef83e634e3b8c154f2c4b2da
Successfully built implicit


In [5]:
from tqdm.auto import tqdm
import numpy as np
import pandas as pd
import random

from scipy.sparse import csr_matrix, vstack

In [6]:
tr = pd.read_json("train.json")
te = pd.read_json("val.json")

tr_songs = tr.songs.tolist()
te_songs = te.songs.tolist()
tr_tags = tr.tags.tolist()
te_tags = te.tags.tolist()
te_ids = te.id.tolist()

In [7]:
import tensorflow as tf
import sys
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
# TF를 백엔드로 사용하는 케라스 환경에서
# 필요에 따라 탄력적으로 GPU 메모리를 사용하도록 설정

session = tf.compat.v1.Session(config=config)

from keras import optimizers
import glob
np.set_printoptions(threshold=np.inf) #...없이 출력하기
import matplotlib.pyplot as plt
from keras.preprocessing.image import load_img, array_to_img, img_to_array, ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Input, BatchNormalization
np.random.seed(111)

# Preprocessing

## Using sparse matrix
### Songs

In [8]:
def unique_item(array1, array2): # array1에 있는 원소들 중 array2에도 있는 것들을 제거하는 함수
    array1 = list(array1)
    for x in array1:
        if x in array2:
            array1.remove(x)
    return array1

In [9]:
tr = []
iid_to_idx = {}
tag_to_idx = {} 
idx = 0

for i, l in enumerate(tr_songs):
    view = l
    for item_id in view:
        if item_id not in iid_to_idx:
            iid_to_idx[item_id] = idx
            idx += 1
    view = [iid_to_idx[x] for x in view]
    tr.append(view)

n_items = len(iid_to_idx)

idx_to_iid = {x:y for(y,x) in iid_to_idx.items()}

te = []

idx = 0
for i, l in enumerate(te_songs):
    view = l
    ret = [] 
    for item_id in view:
        if item_id not in iid_to_idx:
            continue
        ret.append(iid_to_idx[item_id])
    te.append(ret)

In [10]:
rows = np.repeat(0,len(tr[0]))
cols = tr[0]
vals = np.repeat(1,len(tr[0]))
tr_csr = csr_matrix((vals, (rows, cols)),[1, n_items]) # train 데이터의 첫 row를 csr_matrix화

for i in tqdm(range(1, len(tr))): # train 데이터를 stack을 쌓는 방식으로 csr_matrix화
    rows = np.repeat(0,len(tr[i]))
    cols = tr[i]
    vals = np.repeat(1,len(tr[i]))
    new_csr = csr_matrix((vals, (rows, cols)),[1, n_items])
    tr_csr = vstack([tr_csr, new_csr])


rows = np.repeat(0,len(te[0]))
cols = te[0]
vals = np.repeat(1,len(te[0]))
te_csr = csr_matrix((vals, (rows, cols)),[1, n_items]) # test(validation) 데이터의 첫 row를 csr_matrix화

for i in tqdm(range(1, len(te))): # test(validation) 데이터를 stack을 쌓는 방식으로 csr_matrix화
    rows = np.repeat(0,len(te[i]))
    cols = te[i]
    vals = np.repeat(1,len(te[i]))
    new_csr = csr_matrix((vals, (rows, cols)),[1, n_items])
    te_csr = vstack([te_csr, new_csr]) 

song_te_csr= te_csr
song_tr_csr= tr_csr



In [11]:
print("song test csr size:", song_te_csr.shape, "song train csr size:", song_tr_csr.shape)


song test csr size: (23015, 615142) song train csr size: (115071, 615142)


### tags matrix

In [12]:
tr = []
iid_to_idx = {}
tag_to_idx = {} 
idx = 0

for i, l in enumerate(tr_tags):
    view = l
    for item_id in view:
        if item_id not in iid_to_idx:
            iid_to_idx[item_id] = idx
            idx += 1
    view = [iid_to_idx[x] for x in view]
    tr.append(view)

n_items = len(iid_to_idx)

idx_to_iid = {x:y for(y,x) in iid_to_idx.items()}

te = []

idx = 0
for i, l in enumerate(te_tags):
    view = l
    ret = [] 
    for item_id in view:
        if item_id not in iid_to_idx:
            continue
        ret.append(iid_to_idx[item_id])
    te.append(ret)

In [13]:
rows = np.repeat(0,len(tr[0]))
cols = tr[0]
vals = np.repeat(1,len(tr[0]))
tr_csr = csr_matrix((vals, (rows, cols)),[1, n_items]) # train 데이터의 첫 row를 csr_matrix화

for i in tqdm(range(1, len(tr))): # train 데이터를 stack을 쌓는 방식으로 csr_matrix화
    rows = np.repeat(0,len(tr[i]))
    cols = tr[i]
    vals = np.repeat(1,len(tr[i]))
    new_csr = csr_matrix((vals, (rows, cols)),[1, n_items])
    tr_csr = vstack([tr_csr, new_csr])


rows = np.repeat(0,len(te[0]))
cols = te[0]
vals = np.repeat(1,len(te[0]))
te_csr = csr_matrix((vals, (rows, cols)),[1, n_items]) # test(validation) 데이터의 첫 row를 csr_matrix화

for i in tqdm(range(1, len(te))): # test(validation) 데이터를 stack을 쌓는 방식으로 csr_matrix화
    rows = np.repeat(0,len(te[i]))
    cols = te[i]
    vals = np.repeat(1,len(te[i]))
    new_csr = csr_matrix((vals, (rows, cols)),[1, n_items])
    te_csr = vstack([te_csr, new_csr]) 

tag_te_csr= te_csr
tag_tr_csr= tr_csr

In [20]:
print("tag_test csr size:", tag_te_csr.shape, "tag_train csr size:", tag_tr_csr.shape)

test csr size: (23015, 29160) train csr size: (115071, 29160)


## 여기서 부터 train data만 다룸.

# Noising Data for autoencoder
트레인 데이터만 넣는다.

In [93]:
# noising

def noising(csr_mat, noise):
    for i in tqdm(range(csr_mat.shape[0])):
        nonzero_col = csr_mat[i].nonzero()[1]
        nonzero_col_ran = random.sample(list(nonzero_col),int(noise * len(nonzero_col)))
        
        for col in nonzero_col_ran:
            csr_mat[i, col] = 0
        csr_mat.eliminate_zeros() 
    return csr_mat

In [ ]:

# noising song's sparse matrix
song_spa=noising(song_tr_csr, 0.5)

# noising tag's sparse matrix
tag_spa=noising(tag_tr_csr, 0.5)

# Sparse matrix to Sparse Tensor


In [97]:
# sp.sparse.csr_matrix -> tf.SparseTensor로 변환
import scipy as sp

def convert_sparse_matrix_to_sparse_tensor(X):
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    return tf.cast(tf.SparseTensor(indices, coo.data, coo.shape), dtype = tf.float32)

# tf.SparseTensor -> sp.sparse.csr_matrix로 변환
def convert_sparse_tensor_to_sparse_matrix(X):
    coo = sp.sparse.coo_matrix((X.values, 
                                (np.array(X.indices)[:, 0], 
                                 np.array(X.indices)[:, 1])), 
                               shape = np.array(X.shape))
    return coo.tocsr()

In [98]:
song_spa_tensor = convert_sparse_matrix_to_sparse_tensor(song_spa)
tag_spa_tensor = convert_sparse_matrix_to_sparse_tensor(tag_spa)

In [100]:
print(type(song_spa_tensor), type(tag_spa_tensor))

<class 'tensorflow.python.framework.sparse_tensor.SparseTensor'> <class 'tensorflow.python.framework.sparse_tensor.SparseTensor'>


## 엉엉 진짜 모르겠삼

In [96]:
print("song test csr size:", song_spa_tensor.shape, "song train csr size:", song_tr_csr.shape)

print("tag_test csr size:", tag_te_csr.shape, "tag_train csr size:", tag_tr_csr.shape)

song test csr size: (23015, 615142) song train csr size: (115071, 615142)
tag_test csr size: (23015, 29160) tag_train csr size: (115071, 29160)


In [1]:
# 스파스 매트릭스 
from keras.layers import Input, Dense
from keras.models import Model

# 인코딩될 표현(representation)의 크기
encoding_dim = 32

'''32 floats -> 24.5의 압축으로 입력이 784 float라고 가정 '''

# 입력 플레이스홀더
input_img = Input(shape(615142,))  # train_song size

# "encoded"는 입력의 인코딩된 표현
encoded = Dense(encoding_dim, activation='relu')(input_img)

# "decoded"는 입력의 손실있는 재구성 (lossy reconstruction)
decoded = Dense(115071*615142, activation='sigmoid')(encoded)

# 입력을 입력의 재구성으로 매핑할 모델
autoencoder = Model(input_img, decoded)

Using TensorFlow backend.


NameError: ignored

In [ ]:
# 이 모델은 입력을 입력의 인코딩된 입력의 표현으로 매핑
# 2차 인코딩 인듯
encoder = Model(input_img, encoded)


In [ ]:
# 디코더 모델

# 인코딩된 입력을 위한 플레이스 홀더
encoded_input = Input(shape=(encoding_dim,))
# 오토인코더 모델의 마지막 레이어 얻기
decoder_layer = autoencoder.layers[-1]
# 디코더 모델 생성
decoder = Model(encoded_input, decoder_layer(encoded_input))

In [ ]:
# 픽셀마다 = 데이터마다 시그모이드 손실을 사용하도록 모델을 구성하고, 
# optimizer로 Adadelta를 사용합시다.

autoencoder.compile(optimizer='adadelta', loss='sigmoid')
''' loss = "binary_crossentrophy" '''

In [ ]:
# 50번 훈련.
X_train = 
X_test = mnist.load_data()


autoencoder.fit(X_train, X_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(X_test, X_test))

# tf 1.0? autoencoder + sparse matrix
망했지만 참고할 수 있도록!! 남겨둔다

## try

In [56]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.datasets import reuters
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer


In [57]:
max_words = 1000
batch_size = 32
nb_epoch = 5


Loading data...


/usr/local/lib/python3.6/dist-packages/keras/datasets/reuters.py:47: UserWarning: The `nb_words` argument in `load_data` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `load_data` '


2113536/2110848 [==============================] - 2s 1us/step
8982 train sequences
2246 test sequences
46 classes


In [ ]:
type()

In [82]:
X_train = Y_train = song_tr_csr 
X_test = Y_test = song_te_csr 
print(X_train.shape, 'train sequences')
print(X_test.shape, 'test sequences')

(115071, 615142) train sequences
(23015, 615142) test sequences


In [89]:
print('Building model...')
model = Sequential()
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(256))
model.add(Activation('softmax'))

model.compile(loss='sigmoid',
          optimizer='adam',
          metrics=['accuracy'])

history = model.fit(X_train, Y_train,
                nb_epoch=nb_epoch, batch_size=batch_size,
                verbose=1) 
# validation_split=0.1
# score = model.evaluate(X_test, Y_test,
#                       batch_size=batch_size, verbose=1)
# print('Test score:', score[0])
# print('Test accuracy:', score[1])


Building model...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  from ipykernel import kernelapp as app


ValueError: ignored

### random noise



In [22]:
# layer params
# 일단 노래만!
noise_level = 1.0  # 노이즈를 얼마나 줄것인가

#n_inputs = 28 * 28 #인풋사이즈!
n_inputs = song_tr_csr.shape[0] * song_tr_csr.shape[1]
n_hidden1 = 300 
n_hidden2 = 150 # coding units 
n_hidden3 = n_hidden1 
n_outputs = n_inputs


In [23]:
# train params # ################ 
learning_rate = 0.01 
n_epochs = 5 
batch_size = 150 


In [28]:
tf.executing_eagerly()

True

In [26]:
# denoising autoencoder 

inputs = tf.compat.v1.placeholder(tf.float32, shape=[None, n_inputs]) # add gaussian noise 
inputs_noisy = inputs + noise_level * tf.random_normal(tf.shape(inputs)) 
hidden1 = tf.layers.dense(inputs_noisy, n_hidden1, activation=tf.nn.relu, name='hidden1') 
hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name='hidden2') 
hidden3 = tf.layers.dense(hidden2, n_hidden3, activation=tf.nn.relu, name='hidden3') 
outputs = tf.layers.dense(hidden3, n_outputs, name='outputs')



RuntimeError: ignored

In [ ]:
# loss 
reconstruction_loss = tf.losses.mean_squared_error(labels=inputs, predictions=outputs)

# optimizer 
train_op = tf.train.AdamOptimizer(learning_rate).minimize(reconstruction_loss) 

# saver 
saver = tf.train.Saver()


## tf 2.0 autoencoder


In [53]:
from tensorflow import keras
from tensorflow.keras import layers

In [55]:
# add noise(import augmenters)
noise= 0.1

# inputs_noisy = inputs + noise_level * tf.random_normal(tf.shape(inputs)) 

In [45]:
# model
class Encoder(keras.Model):
    def __init__(self, intermediate_dim):
        super(Encoder, self).__init__()
        self.input_layer = keras.layers.Dense(units=original_dim,activation=tf.nn.relu)
        self.hidden_layer = keras.layers.Dense(units=intermediate_dim, activation=tf.nn.relu)
        self.output_layer = keras.layers.Dense(units=intermediate_dim, activation=tf.nn.sigmoid)

    def call(self, inputs):
        x = self.input_layer(inputs)
        x = self.hidden_layer(x)
        return self.output_layer(x)

class Decoder(keras.Model):
    def __init__(self, intermediate_dim, original_dim):
        super(Decoder, self).__init__()
        self.input_layer = keras.layers.Dense(units=intermediate_dim, activation=tf.nn.relu)
        self.hidden_layer = keras.layers.Dense(units=intermediate_dim,activation=tf.nn.relu)
        self.output_layer = keras.layers.Dense(units=original_dim,activation=tf.nn.sigmoid)

    def call(self, code):
        x = self.input_layer(code)
        x = self.hidden_layer(x)
        return self.output_layer(x)



In [46]:
class Autoencoder(keras.Model):
    def __init__(self, intermediate_dim, original_dim):
        super(Autoencoder, self).__init__()
        self.encoder = Encoder(intermediate_dim=intermediate_dim)
        self.decoder = Decoder(intermediate_dim=intermediate_dim,original_dim=original_dim)

    def call(self, inputs):
        code = self.encoder(inputs)
        reconstructed = self.decoder(code)
        return reconstructed



In [47]:


autoencoder = Autoencoder(
    intermediate_dim=intermediate_dim,
    original_dim=original_dim
)

NameError: ignored

In [36]:
# sequential model

model = tf.keras.Sequential()
# 64개의 유닛을 가진 완전 연결 층을 모델에 추가합니다:
model.add(layers.Dense(64, activation='relu'))
# 또 하나를 추가합니다:
model.add(layers.Dense(64, activation='relu'))
# 10개의 출력 유닛을 가진 소프트맥스 층을 추가합니다:
model.add(layers.Dense(10, activation='softmax'))

In [37]:
# 시그모이드 활성화 층을 만듭니다:
layers.Dense(64, activation='sigmoid')
# 또는 다음도 가능합니다:
layers.Dense(64, activation=tf.keras.activations.sigmoid)

# 커널 행렬에 L1 규제가 적용된 선형 활성화 층. 하이퍼파라미터 0.01은 규제의 양을 조절합니다:
layers.Dense(64, kernel_regularizer=tf.keras.regularizers.l1(0.01))

# 절편 벡터에 L2 규제가 적용된 선형 활성화 층. 하이퍼파라미터 0.01은 규제의 양을 조절합니다:
layers.Dense(64, bias_regularizer=tf.keras.regularizers.l2(0.01))

# 커널을 랜덤한 직교 행렬로 초기화한 선형 활성화 층:
layers.Dense(64, kernel_initializer='orthogonal')

# 절편 벡터를 상수 2.0으로 설정한 선형 활성화 층:
layers.Dense(64, bias_initializer=tf.keras.initializers.Constant(2.0))

In [38]:
model = tf.keras.Sequential([
# 64개의 유닛을 가진 완전 연결 층을 모델에 추가합니다:
layers.Dense(64, activation='relu', input_shape=(32,)),
# 또 하나를 추가합니다:
layers.Dense(64, activation='relu'),
# 10개의 출력 유닛을 가진 소프트맥스 층을 추가합니다:
layers.Dense(10, activation='softmax')])

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='sigmoid',
              metrics=['accuracy'])

Tf.keras.Model.compile
- optimizer : 훈련과정 설정, 옵티마이저 설정
- loss : 최적화 과정에서 최소화될 손실함수. mse, crossentrophy 등
- metrics : 훈련을 모니터링하기위해 사용됨. 
- 참고 링크 : 기본 텐서플로 모델 만들기 https://www.tensorflow.org/guide/keras/overview?hl=ko
- 참고 링크 : 오토인코더 텐서플로 2.0 https://blog.naver.com/dbwlsruddus0309/221743817185

